In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.petri.importer import importer as pnml_importer
from pm4py.visualization.petrinet import visualizer as pn_visualizer


# Q4

### Import

In [2]:
no_appeals_log = xes_importer.apply("../../FMP_no_appeals_cleaned.xes")

no_appeals_net, no_appeals_initial_marking, no_appeals_final_marking = pnml_importer.apply("../results/im_no_appeals.pnml")


In [3]:
appeals_log = xes_importer.apply("../../FMP_appeals.xes")

appeals_net, appeals_initial_marking, appeals_final_marking = pnml_importer.apply("../results/im_appeals.pnml")


In [6]:
parameters = {pn_visualizer.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}

no_appeals_performance_viz = pn_visualizer.apply(no_appeals_net, no_appeals_initial_marking, no_appeals_final_marking, parameters=parameters, variant=pn_visualizer.Variants.PERFORMANCE, log=no_appeals_log)
pn_visualizer.save(no_appeals_performance_viz, "no_appeals_performance.png")


In [7]:
appeals_performance_viz = pn_visualizer.apply(appeals_net, appeals_initial_marking, appeals_final_marking, parameters=parameters, variant=pn_visualizer.Variants.PERFORMANCE, log=appeals_log)
pn_visualizer.save(appeals_performance_viz, "appeals_performance.png")
